In [1]:
from RiskMax import Bigmaxcut
import pandas as pd
import numpy as np
import itertools
import os

In [2]:
pd.set_option('max_columns',30)

## Reading the Data

In [4]:
df = pd.read_csv('train_default.csv',sep="|")
df.shape

(8400, 25)

In [5]:
df.rename(columns={"default.payment.next.month":"tgt"},inplace=True)
df.rename(columns={x:"v_"+x for x in ["LIMIT_BAL","AGE","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]},inplace=True)
df.rename(columns={x:"c_"+x for x in df.columns if x not in ["ACCOUNT_ID","tgt"] and x[0]!="v"},inplace=True)

In [6]:
df.head(2)

,ACCOUNT_ID,v_LIMIT_BAL,c_SEX,c_EDUCATION,c_MARRIAGE,v_AGE,c_PAY_0,c_PAY_2,c_PAY_3,c_PAY_4,c_PAY_5,c_PAY_6,v_BILL_AMT1,v_BILL_AMT2,v_BILL_AMT3,v_BILL_AMT4,v_BILL_AMT5,v_BILL_AMT6,v_PAY_AMT1,v_PAY_AMT2,v_PAY_AMT3,v_PAY_AMT4,v_PAY_AMT5,v_PAY_AMT6,tgt
2178,21932,250000.0,1,1,2,27,0,0,0,0,0,0,144525.0,183569.0,186020.0,186465.0,113336.0,0.0,43089.0,6959.0,110000.0,0.0,0.0,132818.0,0
19440,27099,30000.0,2,2,1,48,2,2,2,2,2,0,28695.0,29824.0,29066.0,31236.0,29819.0,30058.0,1900.0,0.0,3000.0,0.0,1500.0,2000.0,1


## Generation of Variables

In [7]:
%%time
df["v_PAY_AMT_TOTAL"] = df["v_PAY_AMT1"]+df["v_PAY_AMT2"]+df["v_PAY_AMT3"]+df["v_PAY_AMT4"]+df["v_PAY_AMT5"]+df["v_PAY_AMT6"]

for x in list(itertools.combinations(["v_LIMIT_BAL","v_AGE","v_BILL_AMT1","v_BILL_AMT2","v_BILL_AMT3","v_BILL_AMT4","v_BILL_AMT5","v_BILL_AMT6","v_PAY_AMT1","v_PAY_AMT2","v_PAY_AMT3","v_PAY_AMT4","v_PAY_AMT5","v_PAY_AMT6","v_PAY_AMT_TOTAL"], 2)):
    df["v_"+x[0][2:]+"_sum_"+x[1][2:]]=df[x[0]]+df[x[1]]
    df["v_"+x[0][2:]+"_ratio_"+x[1][2:]]=df[x[0]]/df[x[1]]
    df["v_"+x[1][2:]+"_mean_"+x[0][2:]]=df[[x[1],x[0]]].apply(lambda x: np.mean(x),axis=1)

CPU times: user 1min 9s, sys: 482 ms, total: 1min 9s
Wall time: 1min 9s


In [8]:
continuas = df.filter(like="v_").columns.tolist()
categoricas = df.filter(like="c_").columns.tolist()

In [9]:
df.shape

(8400, 341)

## Filling NAN and Replacing Inf with 0 due the ratio Variables

In [10]:
df.fillna(0,inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)

## Cut of continious Variables for Woe

In [11]:
%%time
cut=Bigmaxcut(variables=continuas, numero = 3, metodos=["arbol"])
cut.fit(df,"tgt")

variable:  v_LIMIT_BAL
variable:  v_AGE
variable:  v_BILL_AMT1
variable:  v_BILL_AMT2
variable:  v_BILL_AMT3
variable:  v_BILL_AMT4
variable:  v_BILL_AMT5
variable:  v_BILL_AMT6
variable:  v_PAY_AMT1
variable:  v_PAY_AMT2
variable:  v_PAY_AMT3
variable:  v_PAY_AMT4
variable:  v_PAY_AMT5
variable:  v_PAY_AMT6
variable:  v_PAY_AMT_TOTAL
variable:  v_LIMIT_BAL_sum_AGE
variable:  v_LIMIT_BAL_ratio_AGE
variable:  v_AGE_mean_LIMIT_BAL
variable:  v_LIMIT_BAL_sum_BILL_AMT1
variable:  v_LIMIT_BAL_ratio_BILL_AMT1
variable:  v_BILL_AMT1_mean_LIMIT_BAL
variable:  v_LIMIT_BAL_sum_BILL_AMT2
variable:  v_LIMIT_BAL_ratio_BILL_AMT2
variable:  v_BILL_AMT2_mean_LIMIT_BAL
variable:  v_LIMIT_BAL_sum_BILL_AMT3
variable:  v_LIMIT_BAL_ratio_BILL_AMT3
variable:  v_BILL_AMT3_mean_LIMIT_BAL
variable:  v_LIMIT_BAL_sum_BILL_AMT4
variable:  v_LIMIT_BAL_ratio_BILL_AMT4
variable:  v_BILL_AMT4_mean_LIMIT_BAL
variable:  v_LIMIT_BAL_sum_BILL_AMT5
variable:  v_LIMIT_BAL_ratio_BILL_AMT5
variable:  v_BILL_AMT5_mean_LIMIT_B

variable:  v_PAY_AMT2_mean_BILL_AMT5
variable:  v_BILL_AMT5_sum_PAY_AMT3
variable:  v_BILL_AMT5_ratio_PAY_AMT3
variable:  v_PAY_AMT3_mean_BILL_AMT5
variable:  v_BILL_AMT5_sum_PAY_AMT4
variable:  v_BILL_AMT5_ratio_PAY_AMT4
variable:  v_PAY_AMT4_mean_BILL_AMT5
variable:  v_BILL_AMT5_sum_PAY_AMT5
variable:  v_BILL_AMT5_ratio_PAY_AMT5
variable:  v_PAY_AMT5_mean_BILL_AMT5
variable:  v_BILL_AMT5_sum_PAY_AMT6
variable:  v_BILL_AMT5_ratio_PAY_AMT6
variable:  v_PAY_AMT6_mean_BILL_AMT5
variable:  v_BILL_AMT5_sum_PAY_AMT_TOTAL
variable:  v_BILL_AMT5_ratio_PAY_AMT_TOTAL
variable:  v_PAY_AMT_TOTAL_mean_BILL_AMT5
variable:  v_BILL_AMT6_sum_PAY_AMT1
variable:  v_BILL_AMT6_ratio_PAY_AMT1
variable:  v_PAY_AMT1_mean_BILL_AMT6
variable:  v_BILL_AMT6_sum_PAY_AMT2
variable:  v_BILL_AMT6_ratio_PAY_AMT2
variable:  v_PAY_AMT2_mean_BILL_AMT6
variable:  v_BILL_AMT6_sum_PAY_AMT3
variable:  v_BILL_AMT6_ratio_PAY_AMT3
variable:  v_PAY_AMT3_mean_BILL_AMT6
variable:  v_BILL_AMT6_sum_PAY_AMT4
variable:  v_BILL_AMT6_r

In [12]:
%%time
df = cut.transform(df)

CPU times: user 9.22 s, sys: 0 ns, total: 9.22 s
Wall time: 9.22 s


In [13]:
df_iv = pd.DataFrame({"variables_iv":list(cut.champion_iv.keys()),"iv":list(cut.champion_iv.values())}).sort_values(by="iv",ascending=False)

In [14]:
df_iv.sort_values("iv",ascending=False).head(5)

,variables_iv,iv
95,v_PAY_AMT_TOTAL_mean_AGE,0.252582
93,v_AGE_sum_PAY_AMT_TOTAL,0.252582
297,v_PAY_AMT2_sum_PAY_AMT_TOTAL,0.243737
299,v_PAY_AMT_TOTAL_mean_PAY_AMT2,0.243737
309,v_PAY_AMT3_sum_PAY_AMT_TOTAL,0.242484


In [16]:
df.head(3)

,ACCOUNT_ID,v_LIMIT_BAL,c_SEX,c_EDUCATION,c_MARRIAGE,v_AGE,c_PAY_0,c_PAY_2,c_PAY_3,c_PAY_4,c_PAY_5,c_PAY_6,v_BILL_AMT1,v_BILL_AMT2,v_BILL_AMT3,...,v_PAY_AMT4_sum_PAY_AMT6,v_PAY_AMT4_ratio_PAY_AMT6,v_PAY_AMT6_mean_PAY_AMT4,v_PAY_AMT4_sum_PAY_AMT_TOTAL,v_PAY_AMT4_ratio_PAY_AMT_TOTAL,v_PAY_AMT_TOTAL_mean_PAY_AMT4,v_PAY_AMT5_sum_PAY_AMT6,v_PAY_AMT5_ratio_PAY_AMT6,v_PAY_AMT6_mean_PAY_AMT5,v_PAY_AMT5_sum_PAY_AMT_TOTAL,v_PAY_AMT5_ratio_PAY_AMT_TOTAL,v_PAY_AMT_TOTAL_mean_PAY_AMT5,v_PAY_AMT6_sum_PAY_AMT_TOTAL,v_PAY_AMT6_ratio_PAY_AMT_TOTAL,v_PAY_AMT_TOTAL_mean_PAY_AMT6
2178,21932,0.329831,1,1,2,0.179370,0,0,0,0,0,0,0.286898,0.307245,0.360416,...,0.880497,0.00000,0.880497,1.003612,0.000000,1.003612,0.892594,0.000000,0.892594,1.035921,0.000000,1.035921,1.040264,-0.057293,1.040264
19440,27099,-0.636876,2,2,1,-0.203529,2,2,2,2,2,0,-0.030881,-0.051329,-0.046538,...,-0.182584,0.00000,-0.182584,-0.332205,0.000000,-0.332205,-0.141639,0.196164,-0.141639,-0.348933,0.104616,-0.348933,-0.315138,-0.057293,-0.315138
8380,19495,-0.051284,2,1,2,0.000000,0,0,0,0,0,2,-0.030881,-0.051329,-0.046538,...,0.880497,0.18881,0.880497,1.003612,-0.165284,1.003612,0.892594,0.000000,0.892594,0.601164,0.000000,0.601164,1.040264,-0.057293,1.040264
